In [0]:
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import IPython.display
import random
import warnings
import os
from PIL import Image
import pathlib
import csv
# sklearn Preprocessing
from sklearn.model_selection import train_test_split
#Keras
import keras
import warnings
warnings.filterwarnings('ignore')
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD

In [0]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))

category = 'melody Noises'.split()
for g in category:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'./drive/My Drive/Colab Notebooks/Melody detection/{g}'):
        songname = f'./drive/My Drive/Colab Notebooks/Melody detection/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        print(y.shape)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(60417,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(110250,)
(

<Figure size 576x576 with 0 Axes>

In [0]:
songname

'./drive/My Drive/Colab Notebooks/Melody detection/Noises/Recording (online-audio-converter.com)83.mp3'

In [0]:
!pip install split_folders

# We will split data by 80% in training and 20% in the test set.

In [0]:
import split_folders
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
split_folders.ratio('./img_data/', output="./data", seed=1337, ratio=(.8, .2)) # default values

Copying files: 238 files [00:00, 2432.87 files/s]


In [0]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255, # rescale all pixel values from 0-255, so aftre this step all our pixel values are in range (0,1)
        shear_range=0.2, #to apply some random tranfromations
        zoom_range=0.2, #to apply zoom
        horizontal_flip=True) # image will be flipper horiz
test_datagen = ImageDataGenerator(rescale=1./255)

The ImageDataGenerator class has three methods flow(), flow_from_directory() and flow_from_dataframe() to read the images from a big numpy array and folders containing images.
We will discuss only flow_from_directory()

In [0]:
training_set = train_datagen.flow_from_directory(
        './data/train',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        shuffle = False)
test_set = test_datagen.flow_from_directory(
        './data/val',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        shuffle = False )

Found 189 images belonging to 2 classes.
Found 49 images belonging to 2 classes.


In [0]:
model = Sequential()
input_shape=(64, 64, 3)
#1st hidden layer
model.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))
#2nd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))
#Flatten
model.add(Flatten())
model.add(Dropout(rate=0.5))
#Add fully connected layer.
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))
#Output layer
model.add(Dense(2))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 31, 31, 32)        896       
_________________________________________________________________
average_pooling2d_1 (Average (None, 15, 15, 32)        0         
_________________________________________________________________
activation_1 (Activation)    (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
average_pooling2d_2 (Average (None, 7, 7, 64)          0         
_________________________________________________________________
activation_2 (Activation)    (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)             

Compile/train the network using Stochastic Gradient Descent(SGD). Gradient Descent works fine when we have a convex curve. But if we don’t have a convex curve, Gradient Descent fails. Hence, in Stochastic Gradient Descent, few samples are selected randomly instead of the whole data set for each iteration.

In [0]:
epochs = 200
batch_size = 8
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])

In [0]:
#fit model with 50 epoch

model.fit_generator(
        training_set,
        steps_per_epoch=100,
        epochs=50,
        validation_data=test_set,
        validation_steps=200)

Epoch 1/50
100/100 [==============================] - 108s 1s/step - loss: 0.7100 - accuracy: 0.4800 - val_loss: 0.6825 - val_accuracy: 0.9796
Epoch 2/50
100/100 [==============================] - 107s 1s/step - loss: 0.6838 - accuracy: 0.5668 - val_loss: 0.6802 - val_accuracy: 0.8776
Epoch 3/50
100/100 [==============================] - 106s 1s/step - loss: 0.6450 - accuracy: 0.6716 - val_loss: 0.6214 - val_accuracy: 0.8980
Epoch 4/50
100/100 [==============================] - 105s 1s/step - loss: 0.4909 - accuracy: 0.8036 - val_loss: 0.5055 - val_accuracy: 0.8571
Epoch 5/50
100/100 [==============================] - 108s 1s/step - loss: 0.2834 - accuracy: 0.8936 - val_loss: 0.0448 - val_accuracy: 1.0000
Epoch 6/50
100/100 [==============================] - 106s 1s/step - loss: 0.1559 - accuracy: 0.9536 - val_loss: 0.0077 - val_accuracy: 1.0000
Epoch 7/50
100/100 [==============================] - 105s 1s/step - loss: 0.1155 - accuracy: 0.9651 - val_loss: 0.0037 - val_accuracy: 1.0000

In [0]:
#Model Evaluation
model.evaluate_generator(generator=test_set, steps=50)

[1.0756053598015569e-05, 1.0]

In [0]:
test_set.reset()
pred = model.predict_generator(test_set, steps=50, verbose=1)

50/50 [==============================] - 16s 310ms/step


In [0]:
predicted_class_indices=np.argmax(pred,axis=1)

labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions = predictions[:49]
filenames=test_set.filenames

In [0]:
predicted_class_indices

array([0, 0, 0, ..., 1, 1, 1])

In [0]:
print(len(filenames),',',len(predictions))

49 , 49


In [0]:
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

In [0]:
results.head()

,Filename,Predictions
0,Noises/Recording (online-audio-convertercom)10...,Noises
1,Noises/Recording (online-audio-convertercom)11...,Noises
2,Noises/Recording (online-audio-convertercom)13...,Noises
3,Noises/Recording (online-audio-convertercom)25...,Noises
4,Noises/Recording (online-audio-convertercom)28...,Noises


In [0]:
!pip install h5py

In [0]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
